In [56]:
import json

def explorar_chaves(d, nivel = 0):
    if isinstance(d, dict):
        for chave, valor in d.items():
            print("  " * nivel + f"Chave: {chave}")
            
            explorar_chaves(valor, nivel + 1)
    
    elif isinstance(d, list) and len(d) > 0:
        print("  " * nivel + f"Lista com {len(d)} elementos - mostrando chaves do primeiro elemento:")
        
        explorar_chaves(d[0], nivel + 1)

with open('data/annotations.json', 'r') as arquivo_json:
    dados = json.load(arquivo_json)

In [57]:
explorar_chaves(dados)

Chave: info
  Chave: year
  Chave: version
  Chave: description
  Chave: contributor
  Chave: url
  Chave: date_created
Chave: images
  Lista com 1500 elementos - mostrando chaves do primeiro elemento:
    Chave: id
    Chave: width
    Chave: height
    Chave: file_name
    Chave: license
    Chave: flickr_url
    Chave: coco_url
    Chave: date_captured
    Chave: flickr_640_url
Chave: annotations
  Lista com 4784 elementos - mostrando chaves do primeiro elemento:
    Chave: id
    Chave: image_id
    Chave: category_id
    Chave: segmentation
      Lista com 1 elementos - mostrando chaves do primeiro elemento:
        Lista com 158 elementos - mostrando chaves do primeiro elemento:
    Chave: area
    Chave: bbox
      Lista com 4 elementos - mostrando chaves do primeiro elemento:
    Chave: iscrowd
Chave: scene_annotations
  Lista com 4296 elementos - mostrando chaves do primeiro elemento:
    Chave: image_id
    Chave: background_ids
      Lista com 1 elementos - mostrando chaves 

***

In [58]:
import os

coco_annotation_path = 'data/annotations.json'

output_dir = 'labels_yolo/'

os.makedirs(output_dir, exist_ok = True)

with open(coco_annotation_path) as f:
    coco = json.load(f)

categories = {cat['id']: cat['name'] for cat in coco['categories']}
category_mapping = {cat['id']: idx for idx, cat in enumerate(coco['categories'])}

def convert_bbox(size, bbox):
    dw = 1. / size[0]
    dh = 1. / size[1]
    x = bbox[0] + bbox[2] / 2.0  
    y = bbox[1] + bbox[3] / 2.0  
    w = bbox[2]  
    h = bbox[3]  
    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return (x, y, w, h)


image_info = {img['id']: img for img in coco['images']}


for annotation in coco['annotations']:
    image_id = annotation['image_id']
    category_id = annotation['category_id']
    bbox = annotation['bbox']

    image = image_info[image_id]
    file_name = image['file_name']
    width = image['width']
    height = image['height']

    txt_file_path = os.path.join(output_dir, file_name.replace('.jpg', '.txt'))

    os.makedirs(os.path.dirname(txt_file_path), exist_ok = True)

    yolo_bbox = convert_bbox((width, height), bbox)

    yolo_class_id = category_mapping[category_id]

    with open(txt_file_path, 'a') as label_file:
        label_file.write(f"{yolo_class_id} " + " ".join([f"{a:.6f}" for a in yolo_bbox]) + '\n')

print(f"Conversão completa! Anotações salvas em {output_dir}.")

Conversão completa! Anotações salvas em labels_yolo/.


***

In [59]:
import os
import shutil

data_path = 'data'
labels_path = 'labels_yolo'

new_dataset_path = 'dataset_prov'
new_images_path = os.path.join(new_dataset_path, 'images')
new_labels_path = os.path.join(new_dataset_path, 'labels')

os.makedirs(new_images_path, exist_ok = True)
os.makedirs(new_labels_path, exist_ok = True)

def move_files_from_batches(batch_folder, target_folder, extension):
    for batch in os.listdir(batch_folder):
        batch_path = os.path.join(batch_folder, batch)
        
        if os.path.isdir(batch_path):
            for file in os.listdir(batch_path):
                if file.endswith(extension):
                    base_name = os.path.splitext(file)[0]
                    new_file_name = f"{base_name}_{batch}{extension}"
                    
                    source_file = os.path.join(batch_path, file)
                    target_file = os.path.join(target_folder, new_file_name)
                    
                    shutil.copy(source_file, target_file)

move_files_from_batches(data_path, new_images_path, '.jpg')

move_files_from_batches(labels_path, new_labels_path, '.txt')

print("Arquivos reorganizados com sucesso!")


Arquivos reorganizados com sucesso!


***

In [60]:
import os
import shutil
from sklearn.model_selection import train_test_split

prov_images_dir = 'dataset_prov/images'
prov_labels_dir = 'dataset_prov/labels'
output_dir = 'dataset'

train_images_dir = os.path.join(output_dir, 'images', 'train')
val_images_dir = os.path.join(output_dir, 'images', 'val')
train_labels_dir = os.path.join(output_dir, 'labels', 'train')
val_labels_dir = os.path.join(output_dir, 'labels', 'val')

os.makedirs(train_images_dir, exist_ok = True)
os.makedirs(val_images_dir, exist_ok = True)
os.makedirs(train_labels_dir, exist_ok = True)
os.makedirs(val_labels_dir, exist_ok = True)

image_files = []
label_files = []

for img_file in os.listdir(prov_images_dir):
    if img_file.endswith('.jpg'):
        img_path = os.path.join(prov_images_dir, img_file)
        label_path = os.path.join(prov_labels_dir, img_file.replace('.jpg', '.txt'))
        
        if os.path.isfile(label_path):
            image_files.append(img_path)
            label_files.append(label_path)

train_imgs, val_imgs, train_labels, val_labels = train_test_split(
    image_files, label_files, test_size = 0.2, random_state = 42
)

for img_path, label_path in zip(train_imgs, train_labels):
    shutil.copy(img_path, train_images_dir)
    shutil.copy(label_path, train_labels_dir)

for img_path, label_path in zip(val_imgs, val_labels):
    shutil.copy(img_path, val_images_dir)
    shutil.copy(label_path, val_labels_dir)

print("Divisão e cópia de arquivos concluídas com sucesso!")


Divisão e cópia de arquivos concluídas com sucesso!
